In [ ]:
import cv2
import numpy as np
import tensorflow as tf

# Load trained model
model = tf.keras.models.load_model("model/stress_model.h5")

# Load face detector
face_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
)

# Open webcam
cap = cv2.VideoCapture(0,cv2.CAP_DSHOW)
if not cap.isOpened():
    print("Camera not opened")

while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        break
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

for (x, y, w, h) in faces:
    face = gray[y:y+h, x:x+w]

    if face.size == 0:
        continue

    face = cv2.resize(face, (48, 48))
    face = face / 255.0
    face = face.reshape(1, 48, 48, 1)

    prediction = model.predict(face, verbose=0)

    stress_prob = prediction[0][0]

    if stress_prob > 0.5:
        label = f"STRESSED ({stress_prob:.2f})"
        color = (0, 0, 255)
    else:
        label = f"NOT STRESSED ({1 - stress_prob:.2f})"
        color = (0, 255, 0)

    # Draw box
    cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

    # Draw label
    cv2.putText(frame,label,(x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.9,color, 2)
    cv2.imshow("Facial Stress Detection", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()